## Noor Sharma Capstone Project IBM

**Importing all the required libraries**

In [73]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
#=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [74]:
from bs4 import BeautifulSoup

In [75]:
website_url = requests.get("https://en.wikipedia.org/wiki/Regional_Municipality_of_Durham").text

In [76]:
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Regional Municipality of Durham - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Regional_Municipality_of_Durham","wgTitle":"Regional Municipality of Durham","wgCurRevisionId":915707821,"wgRevisionId":915707821,"wgArticleId":254159,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages with reference errors","Pages with duplicate reference names","Articles with short description","Coordinates on Wikidata","Articles to be expanded from June 2008","All articles to be expanded","Articles using small message boxes","All articles with unsourced statements","Articles with unsourced statements from March 2015","Commons category link from Wikidata","Wikipedia articles with ISNI identifiers","

In [77]:
My_table = soup.find('table',{'class':'wikitable'})
My_table

<table class="wikitable" style="text-align:left;">
<tbody><tr>
<th>Municipality</th>
<th>2016 Population<sup class="reference" id="cite_ref-2"><a href="#cite_note-2">[2]</a></sup>
</th></tr>
<tr>
<td>City of <a href="/wiki/Oshawa" title="Oshawa">Oshawa</a></td>
<td>159,458
</td></tr>
<tr>
<td>Town of <a href="/wiki/Whitby,_Ontario" title="Whitby, Ontario">Whitby</a></td>
<td>128,377
</td></tr>
<tr>
<td>Town of <a href="/wiki/Ajax,_Ontario" title="Ajax, Ontario">Ajax</a></td>
<td>119,677
</td></tr>
<tr>
<td>Municipality of <a href="/wiki/Clarington" title="Clarington">Clarington</a></td>
<td>92,013
</td></tr>
<tr>
<td>City of <a href="/wiki/Pickering,_Ontario" title="Pickering, Ontario">Pickering</a></td>
<td>91,771
</td></tr>
<tr>
<td>Township of <a href="/wiki/Scugog" title="Scugog">Scugog</a></td>
<td>21,617
</td></tr>
<tr>
<td>Township of <a href="/wiki/Uxbridge,_Ontario" title="Uxbridge, Ontario">Uxbridge</a></td>
<td>21,176
</td></tr>
<tr>
<td>Township of <a href="/wiki/Brock,_Ont

In [78]:
links = My_table.findAll('a')
links

[<a href="#cite_note-2">[2]</a>,
 <a href="/wiki/Oshawa" title="Oshawa">Oshawa</a>,
 <a href="/wiki/Whitby,_Ontario" title="Whitby, Ontario">Whitby</a>,
 <a href="/wiki/Ajax,_Ontario" title="Ajax, Ontario">Ajax</a>,
 <a href="/wiki/Clarington" title="Clarington">Clarington</a>,
 <a href="/wiki/Pickering,_Ontario" title="Pickering, Ontario">Pickering</a>,
 <a href="/wiki/Scugog" title="Scugog">Scugog</a>,
 <a href="/wiki/Uxbridge,_Ontario" title="Uxbridge, Ontario">Uxbridge</a>,
 <a href="/wiki/Brock,_Ontario" title="Brock, Ontario">Brock</a>]

In [79]:
Durham = []
for link in links:
    Durham.append(link.get('title'))
    
print(Durham)

del Durham[0:1]

Durham

[None, 'Oshawa', 'Whitby, Ontario', 'Ajax, Ontario', 'Clarington', 'Pickering, Ontario', 'Scugog', 'Uxbridge, Ontario', 'Brock, Ontario']


['Oshawa',
 'Whitby, Ontario',
 'Ajax, Ontario',
 'Clarington',
 'Pickering, Ontario',
 'Scugog',
 'Uxbridge, Ontario',
 'Brock, Ontario']

In [80]:
df = pd.DataFrame()
df['Durham'] = Durham
df

,Durham
0,Oshawa
1,"Whitby, Ontario"
2,"Ajax, Ontario"
3,Clarington
4,"Pickering, Ontario"
5,Scugog
6,"Uxbridge, Ontario"
7,"Brock, Ontario"


In [81]:
def get_coords_local(Durham, output_as='center'):
    """
    get the bounding box of a locality in NAD83  given its name

    Parameters
    ----------
    Durham : str
        name of the country in english and lowercase
    output_as : 'str
        chose from 'boundingbox' or 'center'. 
         - 'boundingbox' for [latmin, latmax, lonmin, lonmax]
         - 'center' for [latcenter, loncenter]

    Returns
    -------
    output : list
        list with coordinates as str
    """
    # create url
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search.php?q=',
                             Durham+', Durham Region, Ontario, Canada',
                             '&format=json&polygon=0')
    response = requests.get(url).json()[0]

    # parse response to list
    if output_as == 'boundingbox':
        lst = response[output_as]
        output = [float(i) for i in lst]
    if output_as == 'center':
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
    return output

In [82]:
df2 = df.copy()

latitudeCln = []
longitudeCln = []
for index, row in df2.iterrows():
    print(row[0])
    lat, long = get_coords_local(Durham=row[0], output_as='center')
    latitudeCln.append(lat)
    longitudeCln.append(long)

df2['Latitude'] = latitudeCln
df2['Longitude'] = longitudeCln

df2.shape

Oshawa
Whitby, Ontario
Ajax, Ontario
Clarington
Pickering, Ontario
Scugog
Uxbridge, Ontario
Brock, Ontario


(8, 3)

In [83]:
df

,Durham
0,Oshawa
1,"Whitby, Ontario"
2,"Ajax, Ontario"
3,Clarington
4,"Pickering, Ontario"
5,Scugog
6,"Uxbridge, Ontario"
7,"Brock, Ontario"


In [84]:
df2

,Durham,Latitude,Longitude
0,Oshawa,43.917680,-78.862953
1,"Whitby, Ontario",43.899871,-78.940201
2,"Ajax, Ontario",43.850374,-79.024658
3,Clarington,43.968686,-78.651357
4,"Pickering, Ontario",43.835765,-79.090576
5,Scugog,44.115055,-78.905989
6,"Uxbridge, Ontario",44.107556,-79.127655
7,"Brock, Ontario",44.350079,-79.101226


In [85]:
address = 'Durham Region, Ontario, Canada'
geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The decimal coordinates of Durham region are {}, {}.'.format(latitude, longitude))

The decimal coordinates of Durham region are 44.0310471, -78.949678.


In [86]:
# make a copy of the dataframe to get it simply back if needed
df3 = df2.copy()
df3

,Durham,Latitude,Longitude
0,Oshawa,43.917680,-78.862953
1,"Whitby, Ontario",43.899871,-78.940201
2,"Ajax, Ontario",43.850374,-79.024658
3,Clarington,43.968686,-78.651357
4,"Pickering, Ontario",43.835765,-79.090576
5,Scugog,44.115055,-78.905989
6,"Uxbridge, Ontario",44.107556,-79.127655
7,"Brock, Ontario",44.350079,-79.101226


In [87]:
import folium
from folium.features import DivIcon
import folium
import pandas as pd

In [88]:
map_Durham = folium.Map(location=[latitude, longitude], zoom_start=8)

# add markers to map
for lat, lng, local in zip(df3['Latitude'], df3['Longitude'], df3['Durham']):
    label = '{}'.format(local)
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=15,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3).add_to(map_Durham)  
    
map_Durham.save('durham.html')
map_Durham


In [89]:
  #set variables
radius=5000
LIMIT=500
CLIENT_ID = 'REBHNNAYHW4E0YKM13LNF4GBKB1Q5EBJYQW50PZBQWFDIL2K' # your Foursquare ID
CLIENT_SECRET = 'SUXWYLIW3DGOKO5KAS15P1JKDHLOLIJLZVPXFFI4LXX0CORS' # your Foursquare Secret
VERSION = '20190709' # Foursquare API version
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Durham', 
                  'Durham Latitude', 
                  'Durham Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [90]:
# Use category id 4bf58dd8d48988d16e941735 to only get the fast food restaurants
Durham_venues_fastfood = getNearbyVenues(names=df3['Durham'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d16e941735')
Durham_venues_fastfood.head()

,Durham,Durham Latitude,Durham Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Oshawa,43.917680,-78.862953,Subway,43.919389,-78.850797,Sandwich Place
1,Oshawa,43.917680,-78.862953,KFC,43.923635,-78.874548,Fast Food Restaurant
2,"Whitby, Ontario",43.899871,-78.940201,Subway,43.900628,-78.941704,Sandwich Place
3,"Whitby, Ontario",43.899871,-78.940201,Subway,43.905661,-78.947007,Sandwich Place
4,"Ajax, Ontario",43.850374,-79.024658,DQ Grill & Chill Restaurant,43.843049,-79.027976,Fast Food Restaurant


In [91]:
Durham_venues_fastfood.shape

(34, 7)

In [121]:
Durham_venues_fastfood.to_excel('drhamfastfood.xlsx')

In [93]:
#unction to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Durham'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=8,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=.5).add_to(existingMap)

In [94]:
Map_Durham_venues_fastfood = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(Durham_venues_fastfood, 'red', Map_Durham_venues_fastfood)
folium.Marker([43.835765, -79.090576], popup='<i>Pickering</i>').add_to(Map_Durham_venues_fastfood)
folium.Marker([43.917680, -78.862953], popup='<i>Oshawa</i>').add_to(Map_Durham_venues_fastfood)
folium.Marker([43.899871, -78.940201], popup='<i>Whitby</i>').add_to(Map_Durham_venues_fastfood)
folium.Marker([	43.968686,-78.651357], popup='<i>Clarington</i>').add_to(Map_Durham_venues_fastfood)
folium.Marker([43.850374,-79.024658], popup='<i>Ajax</i>').add_to(Map_Durham_venues_fastfood)
folium.Marker([44.115055,-78.905989], popup='<i>Scugog</i>').add_to(Map_Durham_venues_fastfood)
folium.Marker([44.107556,-79.127655], popup='<i>Uxbridge</i>').add_to(Map_Durham_venues_fastfood)
folium.Marker([44.350079,-79.101226], popup='<i>Brock</i>').add_to(Map_Durham_venues_fastfood)
Map_Durham_venues_fastfood.save('durham_fast.html')
Map_Durham_venues_fastfood


In [95]:
# Use category id 4bf58dd8d48988d13b941735 to only get the High Schools
Durham_venues_highschools = getNearbyVenues(names=df3['Durham'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d13b941735')
Durham_venues_highschools.head()

,Durham,Durham Latitude,Durham Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Oshawa,43.91768,-78.862953,Walter E Harris PS,43.914301,-78.854517,Elementary School
1,Oshawa,43.91768,-78.862953,Long & McQuade Musical Instruments,43.921525,-78.874369,Music Store
2,Oshawa,43.91768,-78.862953,Gordon B Attersley P.S,43.926800,-78.858600,School
3,Oshawa,43.91768,-78.862953,Beau Valley Public School,43.926167,-78.867445,School
4,Oshawa,43.91768,-78.862953,Dr. S.j. Phillips Public School,43.913351,-78.869501,Elementary School


In [96]:
Durham_venues_highschools.shape

(32, 7)

In [122]:
Durham_venues_highschools.to_excel('highschools.xlsx')

In [97]:
map_Durham_venues_highschools = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(Durham_venues_highschools, 'green', map_Durham_venues_highschools)
folium.Marker([43.835765, -79.090576], popup='<i>Pickering</i>').add_to(map_Durham_venues_highschools)
folium.Marker([43.917680, -78.862953], popup='<i>Oshawa</i>').add_to(map_Durham_venues_highschools)
folium.Marker([43.899871, -78.940201], popup='<i>Whitby</i>').add_to(map_Durham_venues_highschools)
folium.Marker([	43.968686,-78.651357], popup='<i>Clarington</i>').add_to(map_Durham_venues_highschools)
folium.Marker([43.850374,-79.024658], popup='<i>Ajax</i>').add_to(map_Durham_venues_highschools)
folium.Marker([44.115055,-78.905989], popup='<i>Scugog</i>').add_to(map_Durham_venues_highschools)
folium.Marker([44.107556,-79.127655], popup='<i>Uxbridge</i>').add_to(map_Durham_venues_highschools)
folium.Marker([44.350079,-79.101226], popup='<i>Brock</i>').add_to(map_Durham_venues_highschools)

map_Durham_venues_highschools.save('high_School.html')
map_Durham_venues_highschools

In [98]:
Durham_venues_uni = getNearbyVenues(names=df3['Durham'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4d4b7105d754a06372d81259')
Durham_venues_uni.head()

,Durham,Durham Latitude,Durham Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Oshawa,43.917680,-78.862953,Durham College SW Wing,43.912488,-78.864840,College Science Building
1,Oshawa,43.917680,-78.862953,Resource Centre,43.914580,-78.873674,Student Center
2,"Whitby, Ontario",43.899871,-78.940201,Liaison College,43.899781,-78.938804,College Administrative Building
3,"Whitby, Ontario",43.899871,-78.940201,St. Matthew the Evangelist Catholic School,43.910332,-78.949243,Community College
4,"Ajax, Ontario",43.850374,-79.024658,York Catholic District School Board,43.857314,-79.031969,College Administrative Building


In [123]:
Durham_venues_uni.to_excel('collegesof durham.xlsx')

In [99]:
map_Durham_venues_uni= folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(Durham_venues_uni, 'brown', map_Durham_venues_uni)
folium.Marker([43.835765, -79.090576], popup='<i>Pickering</i>').add_to(map_Durham_venues_uni)
folium.Marker([43.917680, -78.862953], popup='<i>Oshawa</i>').add_to(map_Durham_venues_uni)
folium.Marker([43.899871, -78.940201], popup='<i>Whitby</i>').add_to(map_Durham_venues_uni)
folium.Marker([	43.968686,-78.651357], popup='<i>Clarington</i>').add_to(map_Durham_venues_uni)
folium.Marker([43.850374,-79.024658], popup='<i>Ajax</i>').add_to(map_Durham_venues_uni)
folium.Marker([44.115055,-78.905989], popup='<i>Scugog</i>').add_to(map_Durham_venues_uni)
folium.Marker([44.107556,-79.127655], popup='<i>Uxbridge</i>').add_to(map_Durham_venues_uni)
folium.Marker([44.350079,-79.101226], popup='<i>Brock</i>').add_to(map_Durham_venues_uni)


map_Durham_venues_uni.save('colleges_durham.html')
map_Durham_venues_uni

In [100]:
Durham_venues_office = getNearbyVenues(names=df3['Durham'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4d4b7105d754a06375d81259')
Durham_venues_office 

,Durham,Durham Latitude,Durham Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Oshawa,43.917680,-78.862953,Hillsdale Estates,43.913813,-78.863269,Medical Center
1,Oshawa,43.917680,-78.862953,Calvary Baptist Church,43.917945,-78.861307,Church
2,Oshawa,43.917680,-78.862953,Hillsdale Terraces,43.914931,-78.863690,Medical Center
3,Oshawa,43.917680,-78.862953,Core21,43.914159,-78.862446,Coworking Space
4,Oshawa,43.917680,-78.862953,United Pentecostal Church,43.918109,-78.861779,Church
5,Oshawa,43.917680,-78.862953,Calvary Baptist,43.911485,-78.861371,Church
6,Oshawa,43.917680,-78.862953,Northminster United Church,43.914900,-78.872300,Auditorium
7,Oshawa,43.917680,-78.862953,Christ Memorial Church,43.911650,-78.866303,Church
8,Oshawa,43.917680,-78.862953,Walter E Harris PS,43.914301,-78.854517,Elementary School
9,Oshawa,43.917680,-78.862953,Little Free Library,43.912761,-78.855122,Library


In [101]:
Durham_venues_office.shape

(183, 7)

In [102]:
map_Durham_venues_office = folium.Map(location=[latitude, longitude], zoom_start=11)
addToMap(Durham_venues_office , 'blue', map_Durham_venues_office)
folium.Marker([43.835765, -79.090576], popup='<i>Pickering</i>').add_to( map_Durham_venues_office)
folium.Marker([43.917680, -78.862953], popup='<i>Oshawa</i>').add_to( map_Durham_venues_office)
folium.Marker([43.899871, -78.940201], popup='<i>Whitby</i>').add_to( map_Durham_venues_office)
folium.Marker([	43.968686,-78.651357], popup='<i>Clarington</i>').add_to( map_Durham_venues_office)
folium.Marker([43.850374,-79.024658], popup='<i>Ajax</i>').add_to( map_Durham_venues_office)
folium.Marker([44.115055,-78.905989], popup='<i>Scugog</i>').add_to( map_Durham_venues_office)
folium.Marker([44.107556,-79.127655], popup='<i>Uxbridge</i>').add_to( map_Durham_venues_office)
folium.Marker([44.350079,-79.101226], popup='<i>Brock</i>').add_to( map_Durham_venues_office)

map_Durham_venues_office.save('office_durham.html') 
map_Durham_venues_office

In [103]:
Durham_venues_shop = getNearbyVenues(names=df3['Durham'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1fd941735')
Durham_venues_shop

,Durham,Durham Latitude,Durham Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Oshawa,43.917680,-78.862953,Rossland Square,43.919180,-78.849970,Shopping Mall
1,"Ajax, Ontario",43.850374,-79.024658,riocan,43.857014,-79.022964,Shopping Mall
2,"Pickering, Ontario",43.835765,-79.090576,Pickering Town Centre,43.836032,-79.086304,Shopping Mall
3,"Pickering, Ontario",43.835765,-79.090576,The Hub,43.838132,-79.089718,Shopping Mall
4,"Pickering, Ontario",43.835765,-79.090576,Santa,43.835392,-79.086043,Shopping Mall
5,"Pickering, Ontario",43.835765,-79.090576,Pickering Town Centre Door 7,43.834898,-79.086397,Shopping Mall
6,"Pickering, Ontario",43.835765,-79.090576,Pickering Town Centre Guest Services,43.835129,-79.086270,Shopping Mall
7,"Pickering, Ontario",43.835765,-79.090576,Epic Hair Salon,43.835942,-79.086426,Shopping Mall


In [104]:
Durham_venues_shop.shape

(8, 7)

In [105]:
map_Durham_venues_shop= folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(Durham_venues_shop, '#008080', map_Durham_venues_shop)
map_Durham_venues_shop

In [106]:
map_Durham_venues_all= folium.Map(location=[latitude, longitude], zoom_start=8)


addToMap (Durham_venues_fastfood, 'red', map_Durham_venues_all)  
addToMap(Durham_venues_highschools, 'green', map_Durham_venues_all)
addToMap(Durham_venues_uni, 'brown', map_Durham_venues_all)
addToMap(Durham_venues_shop, '#008080', map_Durham_venues_all)
addToMap(Durham_venues_office, 'blue', map_Durham_venues_all)



map_Durham_venues_all.save('Durham_venues.html')
map_Durham_venues_all

NameError: name 'Durham_venues_all' is not defined

In [113]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Durham').count()
    
    for n in startDf['Durham']:
        try:
            startDf.loc[startDf['Durham'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Durham'] == n,columnTitle] = 0

In [114]:
df_data = df3.copy()
df_data.rename(columns={'Durham':'Durham'}, inplace=True)
addColumn(df_data, 'Fast Food Restaurant', Durham_venues_fastfood)
addColumn(df_data, 'High Schools',Durham_venues_highschools)
addColumn(df_data, 'Colleges', Durham_venues_uni)
addColumn(df_data, 'Malls', Durham_venues_shop)
addColumn(df_data, 'Offices', Durham_venues_office)
df_data

,Durham,Latitude,Longitude,Fast Food Restaurant,High Schools,Colleges,Malls,Offices
0,Oshawa,43.917680,-78.862953,2.0,5.0,2.0,1.0,28.0
1,"Whitby, Ontario",43.899871,-78.940201,2.0,4.0,2.0,0.0,37.0
2,"Ajax, Ontario",43.850374,-79.024658,13.0,7.0,4.0,1.0,43.0
3,Clarington,43.968686,-78.651357,0.0,0.0,0.0,0.0,0.0
4,"Pickering, Ontario",43.835765,-79.090576,17.0,10.0,6.0,6.0,40.0
5,Scugog,44.115055,-78.905989,0.0,0.0,0.0,0.0,1.0
6,"Uxbridge, Ontario",44.107556,-79.127655,0.0,6.0,1.0,0.0,34.0
7,"Brock, Ontario",44.350079,-79.101226,0.0,0.0,0.0,0.0,0.0


In [115]:
# negative weight, because Jeronimo wants to open a burger joint and thus wants to avoid concurrence as much as possible
weight_restaurant = -1

# positive weight, because high school students are good customers
weight_schools = 2

# positive weight, because uni students are good customers
weight_uni = 2

# positive weight because employees are even better customers
weight_office = 3

# positive weight because shopping customers are even better customers
weight_mall=1

In [116]:
df_weighted = df_data[['Durham']].copy()

In [117]:
df_weighted['Score'] = df_data['Fast Food Restaurant'] * weight_restaurant + df_data['High Schools'] * weight_schools + df_data['Colleges'] * weight_uni + df_data['Malls'] * weight_mall + df_data['Offices'] * weight_office 
df_weighted = df_weighted.sort_values(by=['Score'], ascending=False)
df_weighted

,Durham,Score
4,"Pickering, Ontario",141.0
2,"Ajax, Ontario",139.0
1,"Whitby, Ontario",121.0
6,"Uxbridge, Ontario",116.0
0,Oshawa,97.0
5,Scugog,3.0
3,Clarington,0.0
7,"Brock, Ontario",0.0


In [120]:
df_weighted.to_excel('durhamscore.xlsx')

In [118]:
map_Durham_result = folium.Map(location=[latitude, longitude], zoom_start=10)
folium.Marker([43.835765, -79.090576], popup='<i>Pickering, Ontario</i>').add_to(map_Durham_result)
Durham_win = df3[df3['Durham'] == 'Pickering, Ontario']

for lat, lng, local in zip(Durham_win['Latitude'], Durham_win['Longitude'], Durham_win['Durham']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=100,
        popup=label,
        color='blue',
        fill=False,
        fill_color='blue',
        fill_opacity=0.9).add_to(map_Durham_result) 

addToMap(Durham_venues_fastfood[Durham_venues_fastfood['Durham'] == 'Pickering, Ontario'], 'red', map_Durham_result)
addToMap(Durham_venues_highschools[Durham_venues_highschools['Durham'] == 'Pickering, Ontario'], 'green', map_Durham_result)
addToMap(Durham_venues_uni[Durham_venues_uni['Durham'] == 'Pickering, Ontario'], 'gold', map_Durham_result)
addToMap(Durham_venues_shop[Durham_venues_shop['Durham'] == 'Pickering, Ontario'], 'fuchsia', map_Durham_result)
addToMap(Durham_venues_office[Durham_venues_office['Durham'] == 'Pickering, Ontario'], 'fuchsia', map_Durham_result)

map_Durham_result

### Pickering Locality is the best option for Ravi in order to open his Fast Food Restaurant